In [1]:
# 🔧  Install nothing else – Colab already ships PyTorch & Matplotlib
import torch, torch.nn as nn, matplotlib.pyplot as plt, numpy as np
from ipywidgets import interact, IntSlider

In [2]:
def make_sine_wave(seq_len=50):
    """Return a sine wave of length seq_len + 1"""
    x = torch.linspace(0, 3*np.pi, seq_len+1)
    y = torch.sin(x)
    return y[:-1].unsqueeze(1), y[1:].unsqueeze(1)  # t, t+1

inputs, targets = make_sine_wave()
print("📊 One tiny sine wave sample:\n", inputs.squeeze()[:5])

📊 One tiny sine wave sample:
 tensor([0.0000, 0.1874, 0.3681, 0.5358, 0.6845])


In [3]:
rnn_cell = nn.RNNCell(input_size=1, hidden_size=8)
h = torch.zeros(1, 8)                 # hidden state = memory
x_t = torch.tensor([[0.5]])           # one step input

h_next = rnn_cell(x_t, h)             # ← the core math
print("Memory vector after one step:\n", h_next.detach().numpy().round(2))

Memory vector after one step:
 [[-0.41  0.05  0.05 -0.35 -0.18 -0.19 -0.02 -0.49]]


In [4]:
class TinyRNN(nn.Module):
    def __init__(self, hidden=16):
        super().__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=hidden, batch_first=True)
        self.out = nn.Linear(hidden, 1)

    def forward(self, x):
        out, _ = self.rnn(x)          # out shape = (batch, seq, hidden)
        return self.out(out)          # (batch, seq, 1)

In [5]:
model = TinyRNN()
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), 0.01)

seq_len_slider = IntSlider(min=20, max=100, step=10, value=40)

def train_and_plot(seq_len=40):
    inputs, targets = make_sine_wave(seq_len)
    model.train()
    for epoch in range(200):
        pred = model(inputs.unsqueeze(0))
        loss = criterion(pred.squeeze(), targets.squeeze())
        optim.zero_grad(); loss.backward(); optim.step()

    # Visualization
    model.eval()
    with torch.no_grad():
        pred = model(inputs.unsqueeze(0)).squeeze()
    plt.figure(figsize=(8,3))
    plt.title(f"Length {seq_len} – RNN learns sine wave")
    plt.plot(targets.numpy(), label="True next value")
    plt.plot(pred.numpy(), '--', label="RNN prediction")
    plt.legend(); plt.show()

interact(train_and_plot, seq_len=seq_len_slider)

interactive(children=(IntSlider(value=40, description='seq_len', min=20, step=10), Output()), _dom_classes=('w…

<function __main__.train_and_plot(seq_len=40)>

In [6]:
model = TinyRNN()
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), 0.01)

seq_len_slider = IntSlider(min=20, max=100, step=10, value=40)

def train_and_plot(seq_len=40):
    inputs, targets = make_sine_wave(seq_len)
    model.train()
    for epoch in range(200):
        pred = model(inputs.unsqueeze(0))
        loss = criterion(pred.squeeze(), targets.squeeze())
        optim.zero_grad(); loss.backward(); optim.step()

    # Visualization
    model.eval()
    with torch.no_grad():
        pred = model(inputs.unsqueeze(0)).squeeze()
    plt.figure(figsize=(8,3))
    plt.title(f"Length {seq_len} – RNN learns sine wave")
    plt.plot(targets.numpy(), label="True next value")
    plt.plot(pred.numpy(), '--', label="RNN prediction")
    plt.legend(); plt.show()

interact(train_and_plot, seq_len=seq_len_slider)

interactive(children=(IntSlider(value=40, description='seq_len', min=20, step=10), Output()), _dom_classes=('w…

<function __main__.train_and_plot(seq_len=40)>

In [8]:
def show_memory(seq_len=30):
    inputs, _ = make_sine_wave(seq_len)
    hidden_states = []
    h = torch.zeros(1, 1, 16)
    for t in range(seq_len):
        _, h = model.rnn(inputs[t:t+1].unsqueeze(0), h)
        hidden_states.append(h.squeeze().detach().numpy())   # <- .detach() here
    plt.imshow(np.array(hidden_states).T, aspect='auto', cmap='viridis')
    plt.colorbar()
    plt.title("How the memory vector evolves over time")
    plt.xlabel("time step")
    plt.ylabel("hidden unit")
    plt.show()

In [9]:
def gradient_vanishing_demo(seq_len=80):
    long_in, long_tar = make_sine_wave(seq_len)
    model = TinyRNN(hidden=16)
    optim = torch.optim.Adam(model.parameters(), 0.01)
    grads = []
    for epoch in range(50):
        pred = model(long_in.unsqueeze(0))
        loss = criterion(pred.squeeze(), long_tar.squeeze())
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1e6)
        grads.append(grad_norm.item())
        optim.step(); optim.zero_grad()

    plt.semilogy(grads)
    plt.title("Global gradient norm vs. epoch (long sequence)")
    plt.ylabel("||grad||₂ (log)"); plt.xlabel("epoch")
    plt.show()

interact(gradient_vanishing_demo, seq_len=(40,120,10))

interactive(children=(IntSlider(value=80, description='seq_len', max=120, min=40, step=10), Output()), _dom_cl…

<function __main__.gradient_vanishing_demo(seq_len=80)>

In [10]:
from IPython.display import HTML
HTML("""
<ul style="font-size:18px; line-height:1.7;">
<li><b>RNN</b> = a loop that feeds yesterday's output back in today.</li>
<li><b>Hidden state</b> = the network’s scratchpad memory.</li>
<li><b>Vanishing gradient</b> = memory fades on long trips.</li>
<li><b>LSTM/GRU</b> = add “highways” to keep the signal alive.</li>
</ul>
""")

In [13]:
import nbformat

# Load the notebook
nb = nbformat.read("RNN_visual.ipynb", as_version=4)

# Remove metadata.widgets if it exists
for cell in nb.cells:
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]

# Write the cleaned notebook back to disk
nbformat.write(nb, "RNN_visual_cleaned.ipynb")

FileNotFoundError: [Errno 2] No such file or directory: 'RNN_visual.ipynb'

In [12]:
pip install nbformat

In [14]:
!ls

sample_data


In [15]:
pwd

'/content'